## this is the companion notebbok to the [mimikit : Step-by-step netpune tutorial]()

it shows how to use `mimikit` with [neptune.ai](https://neptune.ai) and can be executed for instance in [colab](https://colab.research.google.com/github/k-tonal/mimikit-notebooks/blob/main/Step-by-step%20Gdrive.ipynb)

# I. Pre-requisites

## Install mimikit on your PC

In [ ]:
pip install mimikit

## Setup neptune on your PC

1. Get your API token
    - go to [neptune.ai](https://neptune.ai)
    - log in
    - in the top corner right, click on your avatar
    - the clik on "Get Your API Token", choose your OS and paste **the second option** to your clipboard
2. On your PC
    - open a terminal
    - type `ls -a ~` and see whether you see a `.bashrc`, a `.bash_profile` or none of those.
    - if there were none, create one with `touch ~/.bashrc`
    - we will now assume you found `.bashrc` - both possibilities are equivalent.
    - now type `nano ~/.bashrc` which opens a text editor in the terminal
    - go to the end of the file and paste the line from neptune there
    - type `Ctrl+O` then `ENTER` to save and finally `Ctrl+X` to close the editor.
    - close the terminal
3. Verify that it worked
    - open a new terminal
    - type `echo $NEPTUNE_API_TOKEN`
    - if your token prints out, then congrats! It worked!
    - if not, check that your `.bashrc` or `.bash_profile` does have the line with the token, and if it does, figure out which file your OS is sourcing when starting a fresh terminal... good luck!

## Prepare neptune

go to [neptune.ai](https://neptune.ai) and create 1 new project where you will store your data and the checkpoint of your model.

For the sake of the tutorial, name it `mimikit-demo`.

## Make a database on your PC and upload it to neptune

In [ ]:
# in a terminal :

freqnet-db "demo_db.h5" --roots "music-directory/" --neptune-path "your-username/mimikit-demo"

# II. In this notebook

## If on colab, install `mimikit`

In [ ]:
!pip install mimikit

## Import classes & functions

In `mimikit`, the [`NeptuneConnector`](https://todo-api-link/) manages every part of the connection with `neptune`.

In [1]:
from mimikit import get_trainer
from mimikit.data import Database
from mimikit.freqnet import freqnet_db, FreqNet

from mimikit import NeptuneConnector

## Configure neptune and define some variables

in the `setup` argument, you can define `key`, `value` pairs where the `values` are in your neptune account. 

Two forms are possible :
    - user/project
    - user/project/exp-id
    
the former results in experiments being created when requested - e.g. a `model` to be trained. The latter allows for specific identifiers - e.g. `dbs`.

In [2]:
nep_con = NeptuneConnector(user="Mimi Kit",
                           # edit next line with the exp id of your db :
                           setup={"db": "mimikit-demo/<DB-EXP-ID>",
                                  "model": "mimikit-demo"})

# checkpoints, logs and audios still need to be stored on the notebook's host system before
# they can be uploaded, so we define a root directory where we will store everything during training :

path_to_model = 'demo_model/'

## Download your database to the notebook's environment

In [ ]:
db = nep_con.download_database("db", "demo_db.h5")

## Define a model and some training options

In [ ]:
model = FreqNet(data_object=db.fft,
                model_dim=1024,
                groups=1,
                n_layers=(4,),
                with_skip_conv=False,
                with_residual_conv=False,
                accum_outputs=0,
                concat_outputs=0,
                pad_input=0)

trainer = get_trainer(root_dir=path_to_model,
                      max_epochs=100,
                      
                      # the epochs at which we will save checkpoints :
                      
                      epochs=[30, 60, 99],
                      
                      # pass the model and the NeptuneConnector to get_trainer :

                      model=model,
                      neptune_connector=nep_con
                     )

## Launch the training (and go to [neptune.ai](https://neptune.ai) to see it getting updated live)

In [ ]:
trainer.fit(model)

## Upload the model to neptune

In [ ]:
nep_con.upload_model("model", model)

## Log some infos about your models

In [ ]:
# If your model just has been trained :

model.neptune_experiment.log_text("data", "music-directory/")

## Generate audios with the model

In [ ]:
from mimikit.utils import audio

prompt = model.random_train_example()
output = model.generate(prompt, n_steps=2048)

audio(output.squeeze().numpy())

## Upload the outputs to neptune

`model.log_audio` uploads the outputs as `.wav` in `audios/` and as `.wav.html` in `audio/`.

The first is for you to have standard audio files. The second allows you to listen to the outputs on neptune.ai.

In [ ]:
model.log_audio("awesome-output", output)

# III. Reload / Restore a trained model

## Download a model

once your notebook stopped, you can download your experiment and load a checkpoint :

In [ ]:
nep_con = NeptuneConnector(user="your_username",
                           # edit the next line with the id of the model you trained 
                          setup={"trained": "demo-mimikit/<MODEL_EXP_ID>"})

nep_con.download_experiment("trained")

## Reload a checkpoint

In [ ]:
epoch = 99

path_to_ckpt = path_to_model + "states/epoch=%i.ckpt" % epoch

model = FreqNet(path_to_ckpt, data_object=db.fft)

model.hparams

### Check out the `NeptuneConnector` [API](https://missing-link/) for a complete list of available methods.